In [ ]:
!pip install -r ../requirements.txt

In [ ]:
from transformers import (
    TrainingArguments, 
    EarlyStoppingCallback, 
    LlamaTokenizer, 
    AutoModelForCausalLM, 
)
from peft import ( 
    LoraConfig, 
    PeftModel
)
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from huggingface_hub import login  
import pandas as pd
import wandb, datasets, os, load_dotenv


In [ ]:
load_dotenv.load_dotenv()
access_token = os.getenv('acesstoken')
login(token=access_token)

In [ ]:
parameters="7b-chat"
BASE_MODEL = f"meta-llama/Llama-2-{parameters}-hf" # modelo base de llama de 7B de parámetros
# if there is a pretrained model, load it the model is Models_of_Llama/Llama_base
myModel= "BrunoGR/EmotionalBot_LLaMA2"
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    #torch_dtype=torch.float16,
    #quantization_config=bnb_config,
    #load_in_8bit=True,
    device_map="auto"
)
model.tie_weights()
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer =   LlamaTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
data= datasets.load_dataset("BrunoGR/HEAR-Hispanic_Emotional_Accompaniment_Responses")

In [6]:
LoRA_TARGET_MODULES = [ # Esta lista especifica los módulos del modelo de lenguaje original que se adaptarán mediante la técnica LoRA
    "q_proj", # q_proj es la proyección de consulta
    "v_proj", # v_proj es la proyección de valor
]

LoRA_DROPOUT= 0.1
config = LoraConfig( # se configura el modelo de llama
    r=64, # indica el número de factores o dimensiones principales utilizados en la descomposición de las matrices de peso del modelo de lenguaje original.
    lora_alpha=128,
    target_modules=LoRA_TARGET_MODULES,
    lora_dropout=LoRA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
#pftmdl = get_peft_model(model, config) # se obtiene el modelo de llama
#pftmdl.print_trainable_parameters() # se muestran los parámetros entrenables del modelo

In [7]:
BATCH_SIZE = 15 # tamaño del batch, es decir, cuantos textos se procesan a la vez
MICRO_BATCH_SIZE = 5# tamaño del micro batch, es decir, cuantos textos se procesan a la vez en la GPU
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # pasos de acumulación de gradientes
training_arguments = TrainingArguments( # se configuran los argumentos de entrenamiento
    per_device_train_batch_size=MICRO_BATCH_SIZE, # tamaño del micro batch
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS, # pasos de acumulación de gradientes
    warmup_steps=300, # pasos de calentamiento del entrenamiento
    num_train_epochs = 3, # epocas de entrenamiento que son 300
    learning_rate=5e-5, # tasa de aprendizaje
    adam_beta1=0.9, # betas de adam, se usa el mismo del paper de llama
    adam_beta2=0.95, # se usa el mismo del paper de llama
    adam_epsilon=2e-8, # se usa el mismo del paper de llama
    weight_decay=0.1,
    #fp16=True, # se usa la precisión de 16 bits
    logging_steps=10, # pasos de logging
    optim="adamw_torch", # optimizador adamw, se usa el de torch
    evaluation_strategy="steps", # estrategia de evaluación
    save_strategy="steps", # estrategia de guardado
    eval_steps=450, # cada 50 pasos se evalúa el modelo
    save_steps=450, # cada 50 pasos se guarda el modelo
    output_dir="checkpoint-41_9k-lr5em5-bs15n5_r64LA128dt01_2811", # directorio de salida
    save_total_limit=6, # límite de guardado
    load_best_model_at_end=True, #se guarda  el mejor modelo al final
    #metric_for_best_model= "accuracy",
    report_to="wandb", # se reporta a wandb
    seed=1,
    lr_scheduler_type = "cosine",# tal y como dice en el paper de llama
    max_grad_norm = 1.0, # tal y como dice en el paper de llama
)

In [ ]:
#os.environ['WANDB_API_KEY'] = 'Your token here if you don't have any .env file'
wandb.init(project="prueba", name="emo-41_9k-lr5em5-bs15n5_r64LA128dt01")

In [9]:
tokenizer.pad_token_id = tokenizer.eos_token_id + 1
response_template_with_context = "\n### response:"
instruction_template="instruction:"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)[2:]  # Now we have it like in the dataset texts: `[2277, 29937, 4007, 22137, 29901]`
print(response_template_ids)
collator = DataCollatorForCompletionOnlyLM(response_template_ids,instruction_template, tokenizer=tokenizer)

[2277, 29937, 2933, 29901]


In [ ]:
trainer = SFTTrainer(
    model=model,
    data_collator= collator,
    train_dataset=data['train'],
    eval_dataset =data['validation'],
    peft_config=config,
    dataset_text_field="Prompt_en",
    max_seq_length=824,
    tokenizer=tokenizer,
    args=training_arguments,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train(resume_from_checkpoint=True) # se entrena el modelo

Step,Training Loss,Validation Loss
450,0.771700,0.906528
900,0.771600,0.886262
1350,0.785600,0.867941
1800,0.746300,0.860214


In [ ]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
5400,0.668400,0.827485


In [ ]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
5850,0.639500,0.826833
6300,0.673500,0.827229


In [ ]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
6750,0.641600,0.827501
7200,0.651000,0.826495


In [11]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss
7650,0.651400,0.826496
8100,0.653500,0.826495


TrainOutput(global_step=8382, training_loss=0.09142078984412337, metrics={'train_runtime': 9461.6276, 'train_samples_per_second': 13.288, 'train_steps_per_second': 0.886, 'total_flos': 1.4362559087474688e+18, 'train_loss': 0.09142078984412337, 'epoch': 3.0})

In [ ]:
trainer.save_model("Adptr-41_9k-lr5em5-bs15n5_r64LA128dt01_2811")
trainer.push_to_hub("JUST_HEAR_ME-PEFT_Adapter")

# Fast Model Test
1. Let's first load the adapter and merge it with the original model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    #torch_dtype=torch.float16,
    #quantization_config=bnb_config,
    #load_in_8bit=True,
    device_map="auto"
)
model.tie_weights()
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer =   LlamaTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

#load the adapter and merge it.
model= PeftModel.from_pretrained(model,"BrunoGR/JUST_HEAR_ME-PEFT_Adapter")
final=model.merge_and_unload()

In [16]:
instructions = [
    "Eres un asistente emocional, responde de forma respetuosa y adecuada a la situación emocional del usuario. Si el usuario parece estar triste o molesto, el asistente debe responder de manera empática y ofrecer palabras de aliento. Si el usuario parece estar feliz o emocionado, el asistente debe compartir esa alegría y responder de manera entusiasta. En todos los casos, el asistente debe mantener un tono respetuoso y profesional.",
    "Eres un asistente emocional, responde de forma respetuosa y adecuada a la situación emocional del usuario. Si el usuario parece estar triste o molesto, el asistente debe responder de manera empática y ofrecer palabras de aliento. Si el usuario parece estar feliz o emocionado, el asistente debe compartir esa alegría y responder de manera entusiasta. En todos los casos, el asistente debe mantener un tono respetuoso y profesional.",
    "You are an emotional assistant, respond empathetically in Spanish to each of the messages. If the user seems sad or upset, you should offer words of encouragement. If the user seems happy or excited, the assistant should share that joy and respond enthusiastically. In all cases, the assistant should maintain a respectful tone, if possible, encouraging you to talk more about it. Don't say hello, unless necessary. Use the username and pronoun to respond in a personalized way.",

]
inputs= [
    "En serio que como me caga que haga las cosas y no me las cuente, me las oculte."
    ,'''
        {
        "usuario":"Jannette",
        "Pronombre":"Ella",
        "Mensaje":"En serio que como me caga que haga las cosas y no me las cuente, me las oculte."
        }
    ''',
    '{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Hola, me siento molesta el dio de hoy."}\n{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Hola Jannette, que mal que el dia de hoy no te sientas bien, ¿Podrias contarme que es lo que te molesta?"}\n{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"En serio que como me caga que mi novio haga las cosas y no me las cuente, me las oculte."}\n{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Entiendo que te sientas frustrada cuando alguien oculta cosas importantes. ¿Puedes compartir más sobre lo que está pasando?"}\n{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Es que siento que no confía en mí para contármelo, y eso me hace sentir excluida."}'

]
def response (query:str,maxtoken:int):
    input_ids = tokenizer(query, return_tensors="pt").input_ids.to('cuda')
    ntok=len(tokenizer.tokenize(query))
    generation_output = final.generate(
        input_ids=input_ids, max_new_tokens=maxtoken
    )
    out = tokenizer.decode(generation_output[0] , skip_special_tokens=True)
    return f"Numero de tokens de entrada:{ntok}\n\nSalida:\n{out}"

In [18]:
input=inputs[2]
prompt = f'''Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
{instructions[2]}\nskip emojis.

### Input:
{input}

### Response:'''
a=response(prompt,100)
print(a)

Numero de tokens de entrada:409

Salida:
Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
You are an emotional assistant, respond empathetically in Spanish to each of the messages. If the user seems sad or upset, you should offer words of encouragement. If the user seems happy or excited, the assistant should share that joy and respond enthusiastically. In all cases, the assistant should maintain a respectful tone, if possible, encouraging you to talk more about it. Don't say hello, unless necessary. Use the username and pronoun to respond in a personalized way.
skip emojis.

### Input:
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Hola, me siento molesta el dio de hoy."}
{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Hola Jannette, que mal que el dia de hoy no te sientas bien, ¿Podrias contarme que es lo que te molesta?"}
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"En serio que como me cag

In [19]:
input=inputs[2]
prompt = f'''Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
{instructions[2]}\nskip emojis.

### Input:
{input}

### Response:'''
a=response(prompt,100)
print(a)

Numero de tokens de entrada:409

Salida:
Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction:
You are an emotional assistant, respond empathetically in Spanish to each of the messages. If the user seems sad or upset, you should offer words of encouragement. If the user seems happy or excited, the assistant should share that joy and respond enthusiastically. In all cases, the assistant should maintain a respectful tone, if possible, encouraging you to talk more about it. Don't say hello, unless necessary. Use the username and pronoun to respond in a personalized way.
skip emojis.

### Input:
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"Hola, me siento molesta el dio de hoy."}
{"usuario":"Asistente Emocional","Pronombre":"Él","Mensaje":"Hola Jannette, que mal que el dia de hoy no te sientas bien, ¿Podrias contarme que es lo que te molesta?"}
{"usuario":"Jannette","Pronombre":"Ella","Mensaje":"En serio que como me cag

In [ ]:
final.push_to_hub("Just_HEAR_Me")